In [1]:
"""
Frequency Analysis
"""

COCO_PATH = '../../data/coco/'
COCO_ANNO_PATH = COCO_PATH + 'annotations/'

# load the data from captions
import json
with open(COCO_ANNO_PATH + 'captions_train2014.json') as f:
    coco_captions = json.load(f)
    
    # build the reverse dictionary
    img_captions = {}
    for img_info in coco_captions['images']:
        mid = str(img_info['id'])
        if not mid in img_captions:
            img_captions[mid] = {}
        img_captions[mid]['image'] = img_info
        
    for cap_info in coco_captions['annotations']:
        mid = str(cap_info['image_id'])
        if not 'annotation' in img_captions[mid]:
            img_captions[mid]['annotation'] = []
            img_captions[mid]['captions'] = ''
        img_captions[mid]['annotation'].append(cap_info)
        img_captions[mid]['captions'] += str(cap_info['caption']) + ' '
        

In [2]:
img_captions['8006']['captions']

'A traffic light hanging from the side of a pole. A stop sign and three street signs attached to a pole. A picture of street signs and lamp carve out the building across the streets structural details . The traffic and street signs are in clear view. Street signs at the corner of Broadway and Wall in NYC '

In [3]:
"""
Import Coco Text
"""
import sys
import os
CWD = os.getcwd()
COCO_TEXT_PATH = CWD+'/../../coco-text/'
sys.path.insert(0, COCO_TEXT_PATH)
import coco_text as ct

ct = ct.COCO_Text(COCO_PATH + 'COCO_Text.json')

ct.loadAnns(ct.getAnnIds(imgIds='8006'))

loading annotations into memory...
0:00:02.879961
creating index...
index created!


[]

In [4]:
import re

def cal_coexist_rate(img_id):
    cap = img_captions[img_id]['captions'].upper()
    print 'Captions is :\n%s\n'%cap

    cap_words = set(re.split('([,.])*\s', cap))
    print '%s\n'%cap_words

    texts = ct.loadAnns(ct.getAnnIds(imgIds=int(img_id)))
    text_words = set()
    coexist_count = 0
    for text in texts:
        if not 'utf8_string' in text:
            continue
        words = text['utf8_string'].upper().split(' ')
        for word in words:
            if word in text_words:
                continue
            else:
                text_words.add(word)

            if word in cap_words:
                print 'Word "%s" in caption.'%word
                coexist_count = coexist_count + 1

    print text_words
    print coexist_count/float(len(set(text_words)))
    return coexist_count/float(len(set(text_words))) 

# cal_coexist_rate('8006')
cal_coexist_rate('296614')

Captions is :
SEVERAL MEN PLAYING BASEBALL ON AN OPEN FIELD. THE BOY IS IN BATTING STANCE TO WAITING. A BASEBALL PLAYER TAKES A HIGH GRIP ON THE BAT AS A CATCHER SCRAMBLES FOR THE BALL. A BASEBALL GAME IN ACTION WITH A MAN AT THE PLATE WITH A BAT. A BASEBALL PLAYER HOLDING A BAT ON A FIELD. 

set(['', 'BOY', 'BAT', 'SCRAMBLES', 'GAME', 'BALL', 'BASEBALL', 'HOLDING', 'SEVERAL', 'OPEN', 'PLAYING', 'FOR', 'WITH', '.', 'TO', 'ACTION', 'GRIP', 'BATTING', 'A', 'CATCHER', 'TAKES', 'STANCE', 'PLATE', 'IS', 'AN', 'HIGH', 'FIELD', 'WAITING', 'AT', 'IN', 'THE', 'ON', 'PLAYER', 'MEN', 'AS', 'MAN', None])

set([u'GV'])
0.0


0.0

In [11]:
import re

coexists = []
for img_id in img_captions.keys():
    cap = img_captions[img_id]['captions'].upper()
    cap_words = set(re.split('([,.])*\s', cap))

    texts = ct.loadAnns(ct.getAnnIds(imgIds=int(img_id)))
    if len(texts) == 0:
        continue

    text_words = set()
    coexist_count = 0
    for text in texts:
        if not 'utf8_string' in text:
            continue
        words = text['utf8_string'].upper().split(' ')
        for word in words:
            if word in text_words:
                continue
            else:
                text_words.add(word)

            if word in cap_words:
                coexist_count = coexist_count + 1

    if len(text_words) == 0:
        continue

    img_captions[img_id]['coexistence_data'] = {
        'count'      : coexist_count,
        'text_count' : len(text_words),
        'rate'       : coexist_count/float(len(text_words))
    }
    
    coexists.append((coexist_count, len(text_words),coexist_count/float(len(text_words))))

# len(coexists)
img_captions['8006']['coexistence_data']

{'count': 2, 'rate': 0.6666666666666666, 'text_count': 3}

In [7]:
import statistics

def get_stat(data):
    print "\n"
    print "Total:\t%d;\t\tMean:\t\t%f;\tNonezero:\t%d"%(len(data), statistics.mean(data), len(filter(lambda x:x!=0, data)))
    print "Median:\t%f;\tMedian(H):\t%f;\tMedian(L):\t%f;"%(statistics.median(data), statistics.median_high(data), statistics.median_low(data))
    print "Max:\t%f;\tMin:\t\t%f"%(max(data), min(data))
    print "\n"

for i in [0,1,2]:
    print "Getting the %d data stat:"%i
    get_stat(map(lambda x:x[i], coexists))


Getting the 0 data stat:


Total:	21536;		Mean:		0.462714;	Nonezero:	5783
Median:	0.000000;	Median(H):	0.000000;	Median(L):	0.000000;
Max:	18.000000;	Min:		0.000000


Getting the 1 data stat:


Total:	21536;		Mean:		4.224415;	Nonezero:	21536
Median:	3.000000;	Median(H):	3.000000;	Median(L):	3.000000;
Max:	131.000000;	Min:		1.000000


Getting the 2 data stat:


Total:	21536;		Mean:		0.111039;	Nonezero:	5783
Median:	0.000000;	Median(H):	0.000000;	Median(L):	0.000000;
Max:	1.000000;	Min:		0.000000




In [ ]:
"""
Demo of the histogram (hist) function with a few features.

In addition to the basic histogram, this demo shows a few optional features:

    * Setting the number of data bins
    * The ``normed`` flag, which normalizes bin heights so that the integral of
      the histogram is 1. The resulting histogram is a probability density.
    * Setting the face color of the bars
    * Setting the opacity (alpha value).

"""
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

num_bins = 20
# the histogram of the data
n, bins, patches = plt.hist(map(lambda x:x[2], coexists), num_bins, facecolor='green', alpha=0.5)

# add a 'best fit' line
plt.xlabel('Coexistence Rate')
plt.ylabel('Frequency')
plt.title('Coexistence Rate Distribution')

# Tweak spacing to prevent clipping of ylabel
plt.subplots_adjust(left=0.15)
plt.show()


KeyboardInterrupt: 

In [123]:
# """
# Frequency chart
# """
# import numpy as np
# import matplotlib.pyplot as plt


# # Pile up the history
# freq = {}
# for i in range(max(map(lambda x:x[2], coexists))):
    

# x1 = np.linspace(0.0, 5.0)
# x2 = np.linspace(0.0, 2.0)

# y1 = np.cos(2 * np.pi * x1) * np.exp(-x1)
# y2 = np.cos(2 * np.pi * x2)

# plt.subplot(2, 1, 1)
# plt.plot(x1, y1, 'yo-')
# plt.title('A tale of 2 subplots')
# plt.ylabel('Damped oscillation')

# plt.subplot(2, 1, 2)
# plt.plot(x2, y2, 'r.-')
# plt.xlabel('time (s)')
# plt.ylabel('Undamped')

# plt.show()


In [19]:
"""
Filter out the max most bunch
"""
# len(img_captions)

high_rates_images = [img_captions[k] for k in img_captions.keys() if ('coexistence_data' in img_captions[k] and img_captions[k]['coexistence_data']['rate'] > 0.9)]
print len(high_rates_images)


1012


In [23]:
high_rates_images[55]['image']

{u'coco_url': u'http://mscoco.org/images/87219',
 u'date_captured': u'2013-11-18 03:29:04',
 u'file_name': u'COCO_train2014_000000087219.jpg',
 u'flickr_url': u'http://farm6.staticflickr.com/5441/7396801392_be93a012b7_z.jpg',
 u'height': 428,
 u'id': 87219,
 u'license': 3,
 u'width': 640}